In [1]:
import pandas as pd
import os
import logging

In [2]:
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

In [3]:
baojiadan_folder = 'D:\\Program Files (x86)\\百度云同步盘\\Dropbox\\-E·J- 2014.5.1\\2015.9.26 燕文\\'

In [4]:
baojiadan_file_list = list()

In [5]:
for file_name in os.listdir(baojiadan_folder):
    if file_name.startswith('广州燕文WISH报价单'):
        baojiadan_file_list.append(file_name)

In [6]:
latest_baojiadan_file_name = baojiadan_file_list[-1]
# latest_baojiadan_file_name

In [7]:
latest_baojiadan_path = os.path.join(baojiadan_folder, latest_baojiadan_file_name)
# latest_baojiadan_path

In [8]:
baojiadan_sheets_dict = pd.read_excel(latest_baojiadan_path, sheetname=None)
# type(baojiadan)

In [9]:
wanted_shipping_service_list = ['Wish燕文航空经济小包-普货',
                                'Wish燕文航空经济小包-特货',
                                'Wish燕文专线平邮小包-普货',
                                'Wish燕文专线平邮小包-特货',
                                'WISH燕文C平邮小包',
                                'Wish燕邮宝平邮-特货',
                                'Wish燕文航空挂号小包-普货',
                                'Wish燕文航空挂号小包-特货',
                                'WISH燕文C挂号小包',
                                'WISH燕文C优先小包',
                                'Wish燕邮宝挂号-特货',
                                'Wish燕文航空易派小包-普货',
                                'Wish燕文航空易派小包-特货',
                                'Wish燕文专线追踪小包-普货',
                                'Wish燕文专线追踪小包-特货',
                                'WISH燕特快-普货',
                                'WISH燕特快-特货'
]

In [10]:
export_path = os.path.join(baojiadan_folder, '报价单','燕文（wish）报价表格.xlsx')
writer = pd.ExcelWriter(export_path)

for service_name in wanted_shipping_service_list:
    if service_name in baojiadan_sheets_dict.keys():
        logging.info(service_name + ' is ready to export.')
        df = baojiadan_sheets_dict[service_name]
        try:
            for c in df.columns:
                for index, r in enumerate(df[c]):
                    if r == '国家':
                        country_cell_location = [index, c]
#                         print(country_cell_location)
                    if r == '价格使用说明:':
                        end_location = [index, c]
#                         print(country_cell_location)
                        break
            df = df[country_cell_location[0]+1:end_location[0]]
            df = df.dropna(how='all')
            df = df[df[country_cell_location[1]].str.len() < 10]
#             df['service_name'] = service_name
            print(df.head(5))
            
#             tiers = [1, 2, 3]
#             for tier in tiers:
#                 this_c_name = 'tier_' + str(tier)
#                 next_c = tier + 1
#                 try:
#                     df[this_c_name + '_base'] = df[[tier]]
#                     df[this_c_name + '_surcharge'] = df[[next_c]]
#                     df[this_c_name + '_total'] = df[[tier]].astype(float) * 0.35 + df[this_c_name + '_surcharge'].astype(float)
#                 except Exception as err:
#                     print(str(err))
            
            kg = 0.35
            tiers = [1, 2, 3]
            
            if service_name == 'Wish燕文航空易派小包-普货':
                tiers = [1]
            
            for tier in tiers:
                this_c_name = 'tier_' + str(tier)
                column_num = 2 * tier - 1
                try:
                    df[this_c_name + '_kg_fee'] = df[[column_num]] * kg
                    df[this_c_name + '_kg_fee'] = df[this_c_name + '_kg_fee'].astype(float)
                    df[this_c_name + '_surcharge'] = df[[column_num + 1]].astype(float)
                    df[this_c_name + '_total'] = df[this_c_name + '_kg_fee'] + df[this_c_name + '_surcharge']
                except Exception as err:
                    print(str(err))
                    
#             df_for_merge = df[['service_name', 0, 'tier_1_total']]
            
#             pd_to_merge.append(df_for_merge)
    
            df.to_excel(writer, sheet_name=service_name, index=False)
            logging.info(service_name + ' exported.')
            # df.to_csv(os.path.join(baojiadan_folder, '报价单', key + '.csv'))
        except Exception as err:
            logging.info(service_name + ' failed to export. ' + str(err))
            pass
        # df.to_csv(os.path.join(baojiadan_folder, '报价单', key + '.xlsx'))
        
writer.save()

 2020-04-25 21:57:21,127 - INFO - Wish燕文航空经济小包-普货 is ready to export.
 2020-04-25 21:57:21,231 - INFO - Wish燕文航空经济小包-普货 exported.
 2020-04-25 21:57:21,232 - INFO - Wish燕文航空经济小包-特货 is ready to export.
 2020-04-25 21:57:21,349 - INFO - Wish燕文航空经济小包-特货 exported.
 2020-04-25 21:57:21,351 - INFO - Wish燕文专线平邮小包-普货 is ready to export.


  Wish燕文航空经济小包-普货 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  返回主目录
4              美国         78        6.5         57        6.3    NaN
5              英国         78        6.3         87        5.5    NaN
6              法国         70        6.5         56        6.8    NaN
7              德国         75        6.5         53          7    NaN
8             西班牙         95          5         57        6.8    NaN
  Wish燕文航空经济小包-特货 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  返回主目录
4              美国         90        8.5         90        8.5    NaN
5              英国         75        8.5         89          5    NaN
6              德国         73        7.5         73        7.5    NaN
7             西班牙         92          5        110        4.5    NaN
8              荷兰         75        7.5         75        7.5    NaN
  Wish燕文专线平邮小包-普货 Unnamed: 1 Unnamed: 2      Unnamed: 3 Unnamed: 4  返回主目录
3              英国       56.5        8.9  0.001-0.149 KG      0.001    NaN
4              英国       

 2020-04-25 21:57:21,388 - INFO - Wish燕文专线平邮小包-普货 exported.
 2020-04-25 21:57:21,389 - INFO - Wish燕文专线平邮小包-特货 is ready to export.
 2020-04-25 21:57:21,427 - INFO - Wish燕文专线平邮小包-特货 exported.
 2020-04-25 21:57:21,428 - INFO - WISH燕文C平邮小包 is ready to export.
 2020-04-25 21:57:21,594 - INFO - WISH燕文C平邮小包 exported.
 2020-04-25 21:57:21,596 - INFO - Wish燕邮宝平邮-特货 is ready to export.


  Wish燕文专线平邮小包-特货 Unnamed: 1 Unnamed: 2      Unnamed: 3 Unnamed: 4  返回主目录
3              英国         50        8.9  0.001-0.149 KG      0.001    NaN
4              英国         80        5.3     0.15-2.0 KG      0.001    NaN
5             西班牙         90          4  0.001-0.149 KG      0.001    NaN
6             西班牙         90          4     0.15-2.0 KG      0.001    NaN
7            爱沙尼亚         80          8  0.001-0.149 KG      0.001    NaN
Could not operate 0.35 with block values can't multiply sequence by non-int of type 'float'
  WISH燕文C平邮小包 Unnamed: 1 Unnamed: 2 Unnamed: 3  返回主目录
3          美国       8.16      78.77      56.79    NaN
4         加拿大       8.84       81.6      56.97    NaN
5          英国       8.86      78.87       56.1    NaN
6          法国        8.7         70      54.14    NaN
7          德国       8.78         75      52.53    NaN
  Wish燕邮宝平邮-特货 Unnamed: 1 Unnamed: 2    Unnamed: 3 Unnamed: 4  返回主目录
3           英国       45.5      10.35  0.001-2.0 KG       0.01    NaN
4 

 2020-04-25 21:57:21,619 - INFO - Wish燕邮宝平邮-特货 exported.
 2020-04-25 21:57:21,621 - INFO - Wish燕文航空挂号小包-普货 is ready to export.
 2020-04-25 21:57:21,737 - INFO - Wish燕文航空挂号小包-普货 exported.
 2020-04-25 21:57:21,738 - INFO - Wish燕文航空挂号小包-特货 is ready to export.
 2020-04-25 21:57:21,841 - INFO - Wish燕文航空挂号小包-特货 exported.
 2020-04-25 21:57:21,842 - INFO - WISH燕文C挂号小包 is ready to export.


  Wish燕文航空挂号小包-普货 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
4              美国         62         14         62         14         62   
5             加拿大         80         21         80         21         80   
6              英国         58         17         58         17         58   
7              法国         90         21         90         21         90   
8              德国         74         16         74         16         74   

  Unnamed: 6  返回主目录  
4         14    NaN  
5         21    NaN  
6         17    NaN  
7         21    NaN  
8         16    NaN  
  Wish燕文航空挂号小包-特货 Unnamed: 1 Unnamed: 2  返回主目录
4              美国         67       14.5    NaN
5             加拿大         65       21.5    NaN
6              英国         54         17    NaN
7              法国         68         21    NaN
8              德国         65         26    NaN
  WISH燕文C挂号小包 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
4          美国         55         20         54         20

 2020-04-25 21:57:21,956 - INFO - WISH燕文C挂号小包 exported.
 2020-04-25 21:57:21,957 - INFO - WISH燕文C优先小包 is ready to export.
 2020-04-25 21:57:21,982 - INFO - WISH燕文C优先小包 exported.
 2020-04-25 21:57:21,984 - INFO - Wish燕邮宝挂号-特货 is ready to export.
 2020-04-25 21:57:22,007 - INFO - Wish燕邮宝挂号-特货 exported.
 2020-04-25 21:57:22,009 - INFO - Wish燕文航空易派小包-普货 is ready to export.
 2020-04-25 21:57:22,031 - INFO - Wish燕文航空易派小包-普货 exported.
 2020-04-25 21:57:22,032 - INFO - Wish燕文航空易派小包-特货 is ready to export.
 2020-04-25 21:57:22,106 - INFO - Wish燕文航空易派小包-特货 exported.
 2020-04-25 21:57:22,107 - INFO - Wish燕文专线追踪小包-普货 is ready to export.
 2020-04-25 21:57:22,128 - INFO - Wish燕文专线追踪小包-普货 exported.
 2020-04-25 21:57:22,129 - INFO - Wish燕文专线追踪小包-特货 is ready to export.
 2020-04-25 21:57:22,152 - INFO - Wish燕文专线追踪小包-特货 exported.
 2020-04-25 21:57:22,153 - INFO - WISH燕特快-普货 is ready to export.
 2020-04-25 21:57:22,180 - INFO - WISH燕特快-普货 exported.
 2020-04-25 21:57:22,180 - INFO - WISH燕特快-特货 is ready to e

  WISH燕文C优先小包 Unnamed: 1 Unnamed: 2      Unnamed: 3 Unnamed: 4  返回主目录
3          美国         65         15    0.001-2.0 KG       0.05    NaN
4         加拿大         65         19    0.001-2.0 KG      0.001    NaN
5          英国         55         18  0.001-0.499 KG      0.001    NaN
6          英国         45         25    0.5-1.999 KG      0.001    NaN
7          英国         45         35      2.0-5.0 KG      0.001    NaN
Could not operate 0.35 with block values can't multiply sequence by non-int of type 'float'
  Wish燕邮宝挂号-特货 Unnamed: 1 Unnamed: 2    Unnamed: 3 Unnamed: 4  返回主目录
3           英国       47.5         30  0.001-2.0 KG      0.001    NaN
4           法国       46.5         18  0.001-2.0 KG      0.001    NaN
5           德国       47.5         21  0.001-2.0 KG      0.001    NaN
6          意大利       43.5         32  0.001-2.0 KG      0.001    NaN
7          西班牙       55.5         20  0.001-2.0 KG      0.001    NaN
Could not operate 0.35 with block values can't multiply sequence by non-in

 2020-04-25 21:57:22,200 - INFO - WISH燕特快-特货 exported.
